In [6]:
import pandas as pd
import numpy as np
from collections import Counter
import re

# languange processing imports
import nltk
nltk.download('words')
# from gensim.corpora import Dictionary
# preprocessing imports
from sklearn.preprocessing import LabelEncoder
nltk.download('averaged_perceptron_tagger')

# model imports
# from gensim.models.ldamulticore import LdaMulticore
# from gensim.models.word2vec import Word2Vec
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
# hyperparameter training imports
from sklearn.model_selection import GridSearchCV

# visualization imports
from IPython.display import display
# import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import base64
import io
%matplotlib inline
# sns.set()  # defines the style of 'the plots to be seaborn style
words = set(nltk.corpus.words.words())
import concurrent.futures


[nltk_data] Downloading package words to /home/adrija/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/adrija/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:
reviews  = pd.read_csv('Reviews_Equal.csv', lineterminator='\n',index_col=0)

In [15]:
reviews.head()

,Text,Employment_Status,text_processed
0,"=when i got ring of honer award, that day is a...",0.0,when i got ring of honer award that day is a ...
1,i got to work on a lot of interesting things i...,1.0,i got to work on a lot of interesting things ...
2,immediate management had some common sense but...,1.0,immediate management had some common sense bu...
3,my typical day was working as receptionist for...,0.0,my typical day was working as receptionist fo...
4,prudential is a good ompany with many good peo...,0.0,prudential is a good ompany with many good pe...


In [ ]:
reviews['text_processed'] = reviews['Text'].map(lambda x: re.sub("[\[\]']", '', x))
reviews['text_processed'] = reviews['text_processed'].map(lambda x: re.sub("[,]", ' ', x))

In [14]:
# reviews = reviews.drop(columns = ['text_processed','tokenized_text','tags','Review_title','Review_Star'])
# reviews = reviews.drop(columns = ['Designation','Location','Date','Helpful_Yes','Helpful_No','Company','lang'])
# reviews = reviews[reviews.Employment_Status.notna()]
reviews = reviews.reset_index()
reviews = reviews.drop(columns = ['index'])
# reviews
# print(reviews.head())
# reviews = reviews.drop(columns=['Text','text_processed','tokenized_text'],axis=1)
reviews.isnull().sum()

Text                 0
Employment_Status    0
text_processed       0
dtype: int64

In [12]:
# Remove punctuation
reviews['text_processed'] = reviews['Text'].map(lambda x: re.sub('[\(\),\.!?=]+', '', x))
# Convert the titles to lowercase
reviews['text_processed'] = reviews['text_processed'].map(lambda x: x.lower())
reviews['text_processed'] = reviews['text_processed'].map(lambda x: ' ' + x + ' ')

In [16]:
def get_good_tokens(sentence):
    replaced_punctation = list(map(lambda token: re.sub('[^0-9A-Za-z!?]+', '', token), sentence))
    removed_punctation = list(filter(lambda token: token, replaced_punctation))
    return removed_punctation

In [17]:
nltk.download('punkt')
executor = concurrent.futures.ProcessPoolExecutor() 
def lda_get_good_tokens(df):
    df['Text'] = df.Text.str.lower()
    df['tokenized_text'] = list(map(nltk.word_tokenize, df.Text))
    df['tokenized_text'] = list(map(get_good_tokens, df.tokenized_text))
#     df['tags'] = list(map(nltk.pos_tag,df.tokenized_text))

lda_get_good_tokens(reviews)

[nltk_data] Downloading package punkt to /home/adrija/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
executor = concurrent.futures.ProcessPoolExecutor() 
reviews['tags'] = list(executor.map(nltk.pos_tag,reviews.tokenized_text))

In [18]:
def remove_stopwords(df):
    """ Removes stopwords based on a known set of stopwords
    available in the nltk package. In addition, we include our
    made up word in here.
    """
    # Luckily nltk already has a set of stopwords that we can remove from the texts.
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords_verbs = ['say', 'get', 'go', 'know', 'may', 'need', 'like', 'make', 'see', 'want', 'come', 
                       'take', 'use', 'would', 'can','de', 'la', 'e', 'que', 'en', 'empresa', 'lo', 'con', 'el',
                       'para','nt','!']
    stopwords = stopwords+stopwords_verbs
    # we'll add our own special word in here 'qwerty'
#     stopwords.append(our_special_word)

    df['stopwords_removed'] = list(map(lambda doc:
                                       [word for word in doc if word not in stopwords],
                                       df['tokenized_text']))

remove_stopwords(reviews)

In [19]:
def stem_words(df):
    lemm = nltk.stem.WordNetLemmatizer()
    df['lemmatized_text'] = list(map(lambda sentence:
                                     list(map(lemm.lemmatize, sentence)),
                                     df.stopwords_removed))

    p_stemmer = nltk.stem.porter.PorterStemmer()
    df['stemmed_text'] = list(map(lambda sentence:
                                  list(map(p_stemmer.stem, sentence)),
                                  df.lemmatized_text))

stem_words(reviews)

In [ ]:
reviews.columns
reviews = reviews.drop(columns=['Review_title','Review_Star','Designation','Location','Date','Text','Helpful_Yes',
                                'Helpful_No','Company','lang','text_processed','tokenized_text','tags'],axis=1)
reviews.to_csv('Reviews_Classification.csv')

In [20]:
def get_strings(review):
    stri = ''
    for token in review:
        stri += ' ' + str(token)
    return stri

In [21]:
import concurrent.futures
text = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    for i,analyse in enumerate(executor.map(get_strings,reviews.stemmed_text)):
#         if(i%1000==0):
#             print(i)
        text.append(str(analyse))
#         print(analyse[0])
reviews['preproci'] = pd.Series(text)

In [23]:
reviews= reviews.drop(columns=['text_processed','tokenized_text','stopwords_removed','lemmatized_text','stemmed_text'])

In [ ]:
get_strings(reviews.stemmed_text[0])

In [24]:
reviews.head()

,Text,Employment_Status,preproci
0,"=when i got ring of honer award, that day is a...",0.0,got ring honer award day memor day work envir...
1,i got to work on a lot of interesting things i...,1.0,got work lot interest thing short span time a...
2,immediate management had some common sense but...,1.0,immedi manag common sens store manag a55 hole...
3,my typical day was working as receptionist for...,0.0,typic day work receptionist interior design d...
4,prudential is a good ompany with many good peo...,0.0,prudenti good ompani mani good peopl quit job...


In [ ]:
# import concurrent.futures
# text = []
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     for i,analyse in enumerate(executor.map(get_strings,reviews.stemmed_text)):
# #         if(i%1000==0):
# #             print(i)
#         text.append(str(analyse))
# #         print(analyse[0])
# reviews['preproci'] = pd.Series(text)

In [26]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

# import pandas, xgboost, numpy, textblob, string
# from keras.preprocessing import text, sequence
# from keras import layers, models, optimizers

In [27]:
# split the dataset into training and validation datasets 
train_x,test_x, train_y, test_y = model_selection.train_test_split(reviews['preproci'], 
                                    reviews['Employment_Status'],test_size = 0.3, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [29]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(reviews['preproci'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xtest_count =  count_vect.transform(test_x)

In [45]:
import autosklearn.classification as classifier
model = classifier.AutoSklearnClassifier(time_left_for_this_task=43200,per_run_time_limit=3600,ml_memory_limit=30000)
model.fit(xtrain_count,train_y)

[WARNING] [2020-07-30 03:09:55,389:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:09:57,444:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:09:59,456:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:10:01,467:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:10:03,478:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of model

[WARNING] [2020-07-30 03:11:13,873:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:11:15,885:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:11:17,896:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:11:19,907:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:11:21,919:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of model

[WARNING] [2020-07-30 03:12:32,319:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:12:34,331:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:12:36,341:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:12:38,353:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:12:40,363:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of model

[WARNING] [2020-07-30 03:13:50,768:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:13:52,779:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:13:54,790:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:13:56,801:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:13:58,813:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of model

[WARNING] [2020-07-30 03:15:09,295:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:15:11,306:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:15:13,316:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:15:15,328:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 03:15:17,340:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of model

...

[WARNING] [2020-07-30 07:23:46,368:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:23:48,379:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:23:50,390:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:23:52,401:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:23:54,411:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of model

[WARNING] [2020-07-30 07:25:04,794:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:25:06,807:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:25:08,817:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:25:10,828:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:25:12,839:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of model

[WARNING] [2020-07-30 07:26:23,245:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:26:25,256:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:26:27,266:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:26:29,277:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of models besides current dummy model: 1. Number of dummy models: 1
[WARNING] [2020-07-30 07:26:31,287:EnsembleBuilder(1):ded23583864904b56cff5581c8e4c186] No models better than random - using Dummy Score!Number of model

/home/adrija/anaconda3/envs/indeed/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


AutoSklearnClassifier(delete_output_folder_after_terminate=True,
                      delete_tmp_folder_after_terminate=True,
                      disable_evaluator_output=False,
                      ensemble_memory_limit=1024, ensemble_nbest=50,
                      ensemble_size=50, exclude_estimators=None,
                      exclude_preprocessors=None, get_smac_object_callback=None,
                      include_estimators=None, include_preprocessors=None,
                      initial_configurations_via_metalearning=25,
                      logging_config=None, max_models_on_disc=50,
                      metadata_directory=None, metric=None,
                      ml_memory_limit=30000, n_jobs=None, output_folder=None,
                      per_run_time_limit=3600, resampling_strategy='holdout',
                      resampling_strategy_arguments=None, seed=1,
                      shared_mode=False, smac_scenario_args=None,
                      time_left_for_this_task=432

In [50]:
predictions = model.predict(xtrain_count)

In [51]:
import sklearn
print(sklearn.metrics.classification_report(np.array(train_y), predictions))

              precision    recall  f1-score   support

           0       0.52      0.99      0.68    350262
           1       0.76      0.03      0.07    327470

    accuracy                           0.53    677732
   macro avg       0.64      0.51      0.38    677732
weighted avg       0.64      0.53      0.39    677732



In [52]:
metrics.accuracy_score(predictions, train_y)

0.5281969864194106

In [20]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(reviews['text_processed'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(test_x)

# # ngram level tf-idf 
# tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
# tfidf_vect_ngram.fit(reviews['preproci'])
# xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
# xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(test_x)

# # characters level tf-idf
# tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
# tfidf_vect_ngram_chars.fit(reviews['preproci'])
# xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
# xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(test_x) 

In [ ]:
from gensim.models import Word2Vec
import gensim
import logging

from numpy import random


import re


wv = gensim.models.KeyedVectors.load_word2vec_format("glove.840B.300d.w2vformat.txt", binary=True)
wv.init_sims(replace=True)


def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    


test_tokenized = test_x.apply(lambda r: w2v_tokenize_text(r)).values
train_tokenized = train_x.apply(lambda r: w2v_tokenize_text(r)).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)


In [ ]:
!python -m gensim.scripts.glove2word2vec --input  ../../glove.840B.300d.txt --output glove.840B.300d.w2vformat.txt

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, test_y)

In [39]:
# model = ensemble.RandomForestClassifier(n_estimators=500,n_jobs=24)
# model.fit(xtrain_count,train_y)
predictions = model.predict(xtrain_count)

In [40]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(train_y, predictions))
print(metrics.accuracy_score(predictions, train_y))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    350262
           1       1.00      1.00      1.00    327470

    accuracy                           1.00    677732
   macro avg       1.00      1.00      1.00    677732
weighted avg       1.00      1.00      1.00    677732

0.9998893367879929


In [28]:
confusion_matrix(test_y,predictions)

array([[106430,  43308],
       [ 98435,  42284]])

In [24]:
test_y

array([0, 0, 1, ..., 1, 1, 0])

In [ ]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xtest_count)
print("NB, Count Vectors: " +str(accuracy))

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: " +str(accuracy))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: "+ str(accuracy))

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: " +str(accuracy))



In [ ]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(max_iter=500,n_jobs=-1), xtrain_count, train_y, xtest_count)
print("LR, Count Vectors: " +str(accuracy))

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(max_iter=500,n_jobs=-1), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: " +str(accuracy))

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(max_iter=500,n_jobs=-1), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR, N-Gram Vectors: "+ str(accuracy))

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(max_iter=500,n_jobs=-1), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: " +str(accuracy))

# accuracy = train_model(linear_model.LogisticRegression(max_iter=500,n_jobs=-1), X_train_word_average, train_y, X_test_word_average)
# print("LR, Word2Vec Vectors: " +str(accuracy))




In [ ]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: " +str(accuracy))

In [ ]:
RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(n_jobs=-1), xtrain_count, train_y, xtest_count)
print("RF, Count Vectors: " +str(accuracy))

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(n_jobs=24), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: " +str(accuracy))

accuracy = train_model(ensemble.RandomForestClassifier(n_jobs=24), X_train_word_average, train_y, X_test_word_average)
print("RF, WordLevel TF-IDF: " +str(accuracy))

In [ ]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(n_jobs=24), xtrain_count.tocsc(), train_y, xtest_count.tocsc())
print("Xgb, Count Vectors: "+ str(accuracy))

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(n_jobs=24), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: "+ str(accuracy))

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(n_jobs=24), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: "+ str(accuracy))

In [ ]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
import tensorflow as tf
model = Sequential()
model.add(Embedding(vocabulary_size, 300, input_length=275, weights=[embedding_matrix], trainable=False))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
# model.add(Dense(256, activation='relu'))
model.add(Dense(128,activation='relu'))
# model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
# model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='adam', metrics=['accuracy'], loss = 'sparse_categorical_crossentropy')
# tf.sparse.reorder(xtrain_tfidf_ngram)
# tf.sparse.reorder(train_y)
model.fit(X_train, y_train, validation_data=(X_test,y_test),
          epochs=200,batch_size=512, shuffle=True)


In [ ]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE

In [ ]:
import matplotlib.pyplot as plt
maxlen = []
avg = 0
maximum = 0
for i in range(0,len(reviews.stopwords_removed)):
    num = len(reviews.stopwords_removed[i].split())
#     print(num)
    maxlen.append(num)
    avg+= num

print(avg/len(maxlen))
print(max(maxlen))
plt.plot(maxlen)


In [ ]:
### Create sequence
vocabulary_size = 250000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(reviews.stopwords_removed)
sequences = tokenizer.texts_to_sequences(reviews.stopwords_removed)
data = pad_sequences(sequences, maxlen=150)
print('Found %d unique words.' % len(tokenizer.word_index))

In [ ]:
reviews = reviews[reviews.stopwords_removed.notna()]
reviews=  reviews.reset_index()
reviews = reviews.drop(columns=['index'])
# reviews.to_csv('Reviews_equal.csv')

In [ ]:
embeddings_index = dict()
f = open('../../glove.840B.300d.txt')
for line in f:
    values = line.split()
    word = ''.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [ ]:
from sklearn.model_selection import train_test_split
y = reviews['Employment_Status']
train = data
# test = data[1200000:]
X_train, X_test, y_train, y_test = train_test_split(train,y, test_size = 0.2, random_state = 42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
## create model
model_glove = Sequential()
# model_glove.add(Embedding(vocabulary_size, 300, input_length=150, weights=[embedding_matrix], trainable=True))
# model_glove.add(Dropout(0.2))
# model_glove.add(Conv1D(64, 5, activation='tanh'))
# model_glove.add(MaxPooling1D(pool_size=4))
# model_glove.add(LSTM(300))
# model_glove.add(Dropout(0.2))
model_glove.add(Dense(256,input_shape=(150,),activation='relu'))
model_glove.add(Dropout(0.2))
model_glove.add(Dense(256,activation='relu'))
model_glove.add(Dropout(0.2))
model_glove.add(Dense(128,activation='tanh'))
model_glove.add(Dropout(0.2))
model_glove.add(Dense(64,activation='relu'))
model_glove.add(Dropout(0.2))
model_glove.add(Dense(16,activation='relu'))
model_glove.add(Dropout(0.2))
# model_glove.add(Dense())
model_glove.add(Dense(2, activation=('softmax')))
model_glove.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(X_train, np.asarray(y_train), validation_data=(X_test,np.asarray(y_test)), epochs = 50, batch_size=512)

In [ ]:
## create model
model_glove = Sequential()
# model_glove.add(Embedding(vocabulary_size, 300, input_length=150, weights=[embedding_matrix], trainable=True))
# model_glove.add(Dropout(0.2))
# model_glove.add(Conv1D(64, 5, activation='tanh'))
# model_glove.add(MaxPooling1D(pool_size=4))
# model_glove.add(LSTM(300))
# model_glove.add(Dropout(0.2))
model_glove.add(Dense(1024,input_shape=(150,),activation='relu'))
model_glove.add(Dropout(0.2))
# model_glove.add(Conv1D(64, 5, activation='relu'))
# model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(Dense(512,activation='relu'))
model_glove.add(Dropout(0.2))
model_glove.add(Dense(256,activation='relu'))
model_glove.add(Dropout(0.2))
model_glove.add(Dense(128,activation='tanh'))
model_glove.add(Dropout(0.2))
model_glove.add(Dense(64,activation='relu'))
model_glove.add(Dropout(0.2))
model_glove.add(Dense(16,activation='relu'))
model_glove.add(Dropout(0.2))
# model_glove.add(Dense())
model_glove.add(Dense(2, activation=('softmax')))
model_glove.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(X_train, np.asarray(y_train), validation_data=(X_test,np.asarray(y_test)), epochs = 50, batch_size=512)

In [ ]:
Y_test = y[1200000:]
Y_test
# model_glove.evaluate(test,np.asarray(Y_test))
# print(test.shape)
# print(Y_test.shape)
y_pred= model_glove.predict(test)

In [ ]:
from keras.layers import Dense, Dropout,LSTM
from keras.models import Sequential
import tensorflow as tf
model = Sequential()
model.add(LSTM(100, input_shape = (1,300), return_sequences=True))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='adam', metrics=['accuracy'], loss = 'categorical_crossentropy')
# tf.sparse.reorder(xtrain_tfidf_ngram)
# tf.sparse.reorder(train_y)
model.fit(X_train_word_average,b, epochs=100,validation_split=0.2,batch_size=256, shuffle=True)


In [ ]:
reviews.Employment_Status.value_counts()


In [ ]:
reviews = reviews[reviews['Employment_Status']==0.0]
reviews = reviews.sample(n=500000)

In [ ]:
reviews_fixed = reviews[reviews['Employment_Status']==1.0]

In [ ]:
reviews = pd.concat([reviews,reviews_fixed])

In [ ]:
reviews = reviews.sample(frac=1)
reviews

In [ ]:
reviews.to_csv('Reviews_Equal.csv')

In [ ]:
word_list = []
for word, i in tokenizer.word_index.items():
    word_list.append(word)

In [ ]:
## Get weights
glove_emds = model_glove.layers[0].get_weights()[0]
## Plotting function
def plot_words(data, start, stop, step):
    trace = go.Scatter(
        x = data[start:stop:step,0], 
        y = data[start:stop:step, 1],
        mode = 'markers',
        text= word_list[start:stop:step]
    )
    layout = dict(title= 't-SNE 1 vs t-SNE 2',
                  yaxis = dict(title='t-SNE 2'),
                  xaxis = dict(title='t-SNE 1'),
                  hovermode= 'closest')
    fig = dict(data = [trace], layout= layout)
    py.iplot(fig)
## Visualize words in two dimensions 
glove_tsne_embds = TSNE(n_components=2).fit_transform(glove_emds)
plot_words(glove_tsne_embds, 0, 2000, 1)

In [ ]:
glove_tsne_embds = TSNE(n_components=2).fit_transform(data[0:100000])

In [ ]:
color = ['red' if i==0 else 'green' for i in y[0:100000] ]
start=0
stop=len(data)
step=1
plt.scatter(glove_tsne_embds[start:stop:step,0], glove_tsne_embds[start:stop:step,1], color=color)